In [1]:
CONTEXT_SIZE = 2 # 依据的单词数
EMBEDDING_DIM = 10 # 词向量的维度
# 我们使用莎士比亚的诗
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

In [2]:
trigram = [((test_sentence[i], test_sentence[i+1]), test_sentence[i+2]) 
            for i in range(len(test_sentence)-2)]

In [3]:
# 建立每个词与数字的编码，据此构建词嵌入
vocb = set(test_sentence) # 使用 set 将重复的元素去掉
word_to_idx = {word: i for i, word in enumerate(vocb)}
idx_to_word = {word_to_idx[word]: word for word in word_to_idx}

In [4]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.autograd import Variable

In [5]:
# 定义模型
class n_gram(nn.Module):
    def __init__(self, vocab_size, context_size=CONTEXT_SIZE, n_dim=EMBEDDING_DIM):
        super(n_gram, self).__init__()
        
        self.embed = nn.Embedding(vocab_size, n_dim)
        self.classify = nn.Sequential(
            nn.Linear(context_size * n_dim, 128),
            nn.ReLU(True),
            nn.Linear(128, vocab_size)
        )
        
    def forward(self, x):
        voc_embed = self.embed(x) # 得到词嵌入
        voc_embed = voc_embed.view(1, -1) # 将两个词向量拼在一起
        out = self.classify(voc_embed)
        return out

In [26]:
net = n_gram(len(word_to_idx))#.to(torch.device("mps")) ## 不知为何，这种情况下mps反而慢了不少。

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=1e-2, weight_decay=1e-5)

In [22]:
import tqdm

In [27]:
for e in tqdm.tqdm(range(100)):
    train_loss = 0
    for word, label in trigram: # 使用前 100 个作为训练集
        word = Variable(torch.LongTensor([word_to_idx[i] for i in word]))#.to(torch.device("mps")) # 将两个词作为输入
        label = Variable(torch.LongTensor([word_to_idx[label]]))#.to(torch.device("mps"))
        # 前向传播
        out = net(word)
        loss = criterion(out, label)

    #     break
    # break
        
        train_loss += loss.data#[0]
        # 反向传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if (e + 1) % 20 == 0:
        print('epoch: {}, Loss: {:.6f}'.format(e + 1, train_loss / len(trigram)))

 33%|█████████████▊                            | 33/100 [00:00<00:00, 81.58it/s]

epoch: 20, Loss: 0.919491


 51%|█████████████████████▍                    | 51/100 [00:00<00:00, 84.43it/s]

epoch: 40, Loss: 0.154804


 69%|████████████████████████████▉             | 69/100 [00:00<00:00, 85.74it/s]

epoch: 60, Loss: 0.095166


 96%|████████████████████████████████████████▎ | 96/100 [00:01<00:00, 86.85it/s]

epoch: 80, Loss: 0.075644


100%|█████████████████████████████████████████| 100/100 [00:01<00:00, 83.85it/s]

epoch: 100, Loss: 0.065355


In [28]:
loss

tensor(0.0130, grad_fn=<NllLossBackward0>)

In [29]:
net = net.eval()

In [38]:
# 测试一下结果
word, label = trigram[19]
print('input: {}'.format(word))
print('label: {}'.format(label))
print()
word = Variable(torch.LongTensor([word_to_idx[i] for i in word]))
out = net(word)
pred_label_idx = int(out.max(1)[1].data[0]) #out.max(1)[1].data[0]
predict_word = idx_to_word[pred_label_idx]
print('real word is {}, predicted word is {}'.format(label, predict_word))

input: ('so', 'gazed')
label: on

real word is on, predicted word is on


7